In [ ]:
# triple loss
best_validation_loss = 100
best_acc_top = 0
best_acc_top_n = 0

for epoch in tqdm(range(EPOCHS), desc="Epochs"):
    running_loss = []
    for step, (anchor_img, positive_img, negative_img) in enumerate(tqdm(train_dl, desc="Training", leave=False)):
        anchor_out = cvt_feature_vit_gg.process_extract(anchor_img)
        positive_out = cvt_feature_vit_gg.process_extract(positive_img)
        negative_out = cvt_feature_vit_gg.process_extract(negative_img)
        
        loss = criterion(anchor_out, positive_out, negative_out)
        
        Optimizer.zero_grad()
        loss.backward()
        Optimizer.step()
        running_loss.append(loss.cpu().detach().numpy())
    
    x_trans = convert_feature_transformer(vit_gg, processor_vit_gg, layer="last_hidden_state", row=0, device=DEVICE)
    validation_loss = validate(x_trans)
    acc_top = 
    acc_top_n = 
    
    if acc_top > best_acc_top:
        best_acc_top = acc_top
        save_weight(epoch, cvt_feature_vit_gg, Optimizer, running_loss, acc_top, acc_top_n)
        print("Save model at Best acc top: {:.4f}".format(best_acc_top))
        
    if acc_top_n > best_acc_top_n:
        best_acc_top_n = acc_top_n
        save_weight(epoch, cvt_feature_vit_gg, Optimizer, running_loss, acc_top, acc_top_n)
        print("Save model at Best acc top n: {:.4f}".format(best_acc_top_n))
    
    if validation_loss < best_validation_loss:
        best_validation_loss = validation_loss  # Update the best performance
        save_weight(epoch, cvt_feature_vit_gg, Optimizer, running_loss, acc_top, acc_top_n)
        print("Save model at Best performance: {:.4f}".format(best_validation_loss))
    
    if epoch%10 == 9:
        save_weight(epoch, cvt_feature_vit_gg, Optimizer, running_loss, acc_top, acc_top_n)
        print("Save model at epoch {}".format(epoch+1))

    print("Epoch: {}/{} — Loss: {:.4f} — validation_loss : {:.4f} — acc_top : {} — acc_top_n : {}".format(
        epoch+1, EPOCHS, np.mean(running_loss), validation_loss, acc_top, acc_top_n))